<a href="https://colab.research.google.com/github/hayleypc/HawaiiClimate/blob/main/2024_NCSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fiona

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 33.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import geopandas as gpd
import os
import fiona

In [15]:
# Define file ID and destination path

# https://drive.google.com/drive/folders/1scHx6yRSAgkewSOvzHlktytWaEpLhlsY?usp=drive_link
file_id = '1scHx6yRSAgkewSOvzHlktytWaEpLhlsY'
destination_path = '/content/ncss_labdata.gpkg'

# Download the file
!gdown --id $file_id -O $destination_path

gdf = gpd.read_file(destination_path)

print(gdf.head())


/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gdown/download.py:33: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs4.BeautifulSoup(line, features="html.parser")
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1scHx6yRSAgkewSOvzHlktytWaEpLhlsY

but Gdown can't. Please check connections and permissions.


DataSourceError: /content/ncss_labdata.gpkg: No such file or directory

In [6]:
# https://drive.google.com/drive/u/0/folders/1oZ_ePheGoa_OX2a4a5zz8wPxSrajOpby
pedon_data_path = '/content/drive/MyDrive/hawaii_soils/NCSS US Soil Survey Data/extracted/NASIS_Morphological_09142021.gdb'

soil_data_path = '/content/drive/MyDrive/hawaii_soils/NCSS US Soil Survey Data/extracted/ncss_labdata.gpkg'

❌ File NOT found! Check your file path.


Order: lab_combine_nasis_ncss -> lab_pedon -> lab_layer -> lab_chemical_properties

pedon_key ->

In [11]:
# List available layers
with fiona.Env():
    with fiona.open(soil_data_path, 'r') as src:
        available_layers = src.listlayers()

print("Available layers:", available_layers)


DriverError: Failed to open dataset (flags=68): /content/drive/MyDrive/hawaii_soils/NCSS US Soil Survey Data/extracted/ncss_labdata.gpkg

In [9]:
lab_combine_nasis_ncss = gpd.read_file(soil_data_path, layer = 'lab_combine_nasis_ncss')
lab_pedon = gpd.read_file(soil_data_path, layer = 'lab_pedon')
lab_layer = gpd.read_file(soil_data_path, layer = 'lab_layer')
lab_chemical_properties = gpd.read_file(soil_data_path, layer = 'lab_chemical_properties')


DataSourceError: /content/drive/MyDrive/hawaii_soils/NCSS US Soil Survey Data/extracted/ncss_labdata.gpkg: No such file or directory

In [ ]:
[print(i) for i in lab_chemical_properties.columns]

In [ ]:
lab_combine_nasis_ncss[['pedon_key','geometry']]

lab_pedon[['pedon_key','cntrl_depth_to_top','cntrl_depth_to_bot']]

lab_layer[['pedon_key','layer_key','layer_sequence','layer_type','hzn_top','hzn_bot']]

lab_chemical_properties[["layer_key", "labsampnum", "organic_carbon_walkley_black","oc_walkley_black_method","estimated_organic_carbon","carbon_to_nitrogen_ratio"]]

In [ ]:
# combined_1

In [ ]:
# Step 1: Join lab_combine_nasis_ncss and lab_pedon
combined_1 = pd.merge(
    lab_combine_nasis_ncss[lab_combine_nasis_ncss['state_key'] == 3962][['pedon_key', 'geometry']],
    lab_pedon[['pedon_key', 'cntrl_depth_to_top', 'cntrl_depth_to_bot']],
    on='pedon_key',
    how='inner'
)

# Step 2: Join with lab_layer
combined_2 = pd.merge(
    combined_1,
    lab_layer[['pedon_key', 'layer_key', 'layer_sequence', 'layer_type', 'hzn_top', 'hzn_bot']],
    on='pedon_key',
    how='inner'
)

# Step 3: Join with lab_chemical_properties
final_combined = pd.merge(
    combined_2,
    lab_chemical_properties[[
        'layer_key', 'labsampnum', 'organic_carbon_walkley_black',
        'oc_walkley_black_method', 'estimated_organic_carbon',
        'carbon_to_nitrogen_ratio'
    ]],
    on='layer_key',
    how='inner'
)

# Display or use the final combined dataframe
print(final_combined)
final_combined.to_file('result.gpkg')